In [20]:
from __future__ import print_function
import glob
import time
import pandas as pd
from lxml import etree
from nltk import ngrams
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

In [8]:
%matplotlib inline

In [9]:

t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.05556511878967285  seconds 


In [10]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [11]:
def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object

In [12]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [13]:
def read_xml(li):
  all_objects=[]
  for i in range(len(li)):
    # print("i will run ", i+1," times")
    name_of_file=clean_file_names(li[i])
    # print(name_of_file)
    data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

    post_text='  '
    tree = etree.parse(li[i], parser=parser)
    root = tree.getroot()
    for element in root.iter():
        if element.tag == 'post':
            element_text = element.text
            post_text = post_text + element_text
    mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
    mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



    data_object['posts']=mystring   
    all_objects.append(data_object)

  return all_objects



In [14]:
len(li)

14600

In [15]:
t1=time.time()
data=read_xml(li[0:10000])
t2=time.time()

In [16]:
print("time taken to read :: ", (t2-t1)/100)

time taken to read ::  0.07588720560073853


In [17]:
df=pd.DataFrame(data)

In [18]:
df.tail()

,id,gender,age,category,horoscope,posts
9995,761028,male,27,indUnk,Capricorn,Hey everybody. It's almost 9 here on Sunday n...
9996,3052887,male,24,Engineering,Libra,Mom woke me up with a phone call at 10am! I ...
9997,3371295,female,16,indUnk,Virgo,Wow. So I went shopping with Tashtash today ...
9998,3947895,female,24,Law,Cancer,I have had a special request for blogging so h...
9999,3033260,male,17,indUnk,Leo,"Well, I'm not sure exactly how it happened, wh..."


In [19]:
data=''

## Multilabel classification

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [28]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [29]:
from sklearn.feature_extraction.text import CountVectorizer


In [30]:
sentences = df['posts'].values

In [31]:
labels=df['horoscope']

In [32]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [33]:
le.classes_

array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo'],
      dtype=object)

In [34]:
y=le.transform(labels)

In [40]:
vectorizer = CountVectorizer()
vectorizer.fit(x_train)

x_train = vectorizer.transform(x_train)
x_test  = vectorizer.transform(x_test)

In [127]:
# set parameters:
max_features = 4000
maxlen = 8000
batch_size = 4
embedding_dims = 20
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [128]:
print('Loading data...')
x_train, x_test, y_train,  y_test = train_test_split(sentences,y, test_size=0.2, random_state=42)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
8000 train sequences
2000 test sequences


In [129]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [130]:

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [131]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 8000, 20)          80000     
_________________________________________________________________
dropout_14 (Dropout)         (None, 8000, 20)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 7998, 250)         15250     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 250)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 250)               62750     
_________________________________________________________________
dropout_15 (Dropout)         (None, 250)               0         
_________________________________________________________________
activation_13 (Activation)   (None, 250)               0         
__________

In [138]:


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

ValueError: Error when checking input: expected embedding_8_input to have shape (8000,) but got array with shape (1,)

In [133]:
x_train.shape=(1600*5,)

In [134]:
y_train.shape=(1600*5,)

In [135]:
x_test.shape=(400*5,)

In [136]:
x_test.shape=(400*5,)

In [137]:
x_test.shape

(2000,)

In [100]:
y_test.shape=(400*5,)


In [119]:
x_train.shape

(8000,)

In [139]:
x_train[0]

'  â\x80\x9cWithout a strong commitment to diversity from the worldâ\x80\x99s leading academic institutions, it will become more and more difficult for multinational corporations to compete at the global level.â\x80\x9d  (James Hackett, CEO of Steelcase Inc)  Source:Knickerbocker, Brad.  â\x80\x9cAffirmative Actionâ\x80\x99s Unlikely Allyâ\x80\x9d. Oct. 25, 2000.  Christian Science Monitor,  Kellogg Co. stated that, â\x80\x9cIt will limit its support of campuses that donâ\x80\x99t continue to enroll and graduate growing numbers of ethnic minorities.â\x80\x9d Source: France, Mike and Symonds, William C. â\x80\x9cDiversity is About to Get More Elusive, Not Less.â\x80\x9d July 7 2003 Business WeekurlLink Educating for a Diverse America: A Summit and SymposiumUniversity of Texas, Austin January 29-30, 2004Remarks by President Michael M. Crow :  ......."So it is not unrealistic to start thinking about what change we want 50 years from now. In 2054 the very concept of diversity will change a